![caption](https://www.codeproject.com/KB/database/Visual_SQL_Joins/INNER_JOIN.png)
## Filter / Slice data from weather stations between sunset and sunrise
* Datetime index weather and night_light / sunrise_sunset Data
* Options
    * [slice by date_time](https://www.reddit.com/r/learnpython/comments/4o6bsp/how_do_i_slice_a_pandas_data_frame_according_to_a/)
    * Inner join sunrise_df with sunrise_df

### Load Tools

In [3]:
import pandas as pd
import os.path

### Function for saving results

In [4]:
def save_results(weather_night):
    filepath = '../output/weather_night-180801.csv'
    if os.path.isfile(filepath):
        with open(filepath, 'a') as f:
            weather_night.to_csv(f, header=False, index=False)
    else:
        weather_night.to_csv(filepath, index=False)

### Load weather data

In [5]:
# Load weather station data
weather_src = '../output/weather-170701_171031.csv'
weather_df = pd.read_csv(weather_src, parse_dates=['utc_date'])

In [38]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 727764 entries, 0 to 727763
Data columns (total 23 columns):
utc_date         727764 non-null datetime64[ns]
station_id       727764 non-null object
dewpti           727764 non-null float64
dewptm           727764 non-null float64
heatindexi       727764 non-null float64
heatindexm       727764 non-null float64
hum              727764 non-null int64
precip_ratei     727764 non-null float64
precip_ratem     727764 non-null float64
precip_totali    727764 non-null float64
precip_totalm    727764 non-null float64
pressurei        727764 non-null float64
pressurem        727764 non-null float64
tempi            727764 non-null float64
tempm            727764 non-null float64
wdird            727764 non-null int64
wdire            727764 non-null object
wgusti           727764 non-null float64
wgustm           727764 non-null float64
windchilli       727764 non-null float64
windchillm       727764 non-null float64
wspdi            727764 non

In [39]:
weather_df.head(1)

,utc_date,station_id,dewpti,dewptm,heatindexi,heatindexm,hum,precip_ratei,precip_ratem,precip_totali,...,tempi,tempm,wdird,wdire,wgusti,wgustm,windchilli,windchillm,wspdi,wspdm
0,2017-07-01 06:12:00,KMTCORVA9,47.5,8.6,-9999.0,-9999.0,66,-99.99,-2539.7,0.0,...,58.8,14.9,112,ESE,-999.0,-1607.4,-999.0,-999.0,0.0,0.0


### Load night_light / sunrise_sunset data

In [6]:
# Load sunrise_sunset data
sun_src = '../output/sunrise_sunset-utc_append.csv'
sun_df = pd.read_csv(sun_src, parse_dates=['astronomical_twilight_begin', 'astronomical_twilight_end',
       'civil_twilight_begin', 'civil_twilight_end', 'date', 'day_length',
       'nautical_twilight_begin', 'nautical_twilight_end', 'solar_noon',
       'station_id', 'sunrise', 'sunset'], index_col=[4])
sun_df['utc_date'] = sun_df.index

In [8]:
sun_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4216 entries, 2017-06-30 to 2017-10-31
Data columns (total 12 columns):
astronomical_twilight_begin    4216 non-null datetime64[ns]
astronomical_twilight_end      4216 non-null datetime64[ns]
civil_twilight_begin           4216 non-null datetime64[ns]
civil_twilight_end             4216 non-null datetime64[ns]
day_length                     4216 non-null object
nautical_twilight_begin        4216 non-null datetime64[ns]
nautical_twilight_end          4216 non-null datetime64[ns]
solar_noon                     4216 non-null datetime64[ns]
station_id                     4216 non-null object
sunrise                        4216 non-null datetime64[ns]
sunset                         4216 non-null datetime64[ns]
utc_date                       4216 non-null datetime64[ns]
dtypes: datetime64[ns](10), object(2)
memory usage: 428.2+ KB


In [37]:
sun_df.head(1)

,astronomical_twilight_begin,astronomical_twilight_end,civil_twilight_begin,civil_twilight_end,day_length,nautical_twilight_begin,nautical_twilight_end,solar_noon,station_id,sunrise,sunset,utc_date
date,,,,,,,,,,,,
2017-06-30,2017-06-30 09:05:17,2017-07-01 06:14:50,2017-06-30 11:09:41,2017-07-01 04:10:26,56621,2017-06-30 10:17:42,2017-07-01 05:02:26,2017-06-30 19:40:04,KMTCORVA9,2017-06-30 11:48:13,2017-07-01 03:31:54,2017-06-30


### Go through DataFrame row by row
* retrieve weather_df row 'station_id' and 'utc_date
* if row datetime is in sunset to sunrise range, keep it

In [35]:
def light_weather(sun_df, weather_df):
    # Setup empty DataFrame with weather_df columns
    weather_night = pd.DataFrame(data=None, columns=weather_df.columns)
    
    for index, row in weather_df[:1].iterrows():
        weather_station = row['station_id']
        weather_date = row['utc_date']
        
        print('*****')
        print('Station :', weather_station, 'Index :', index)
        print('Weatherstation DateTime:', weather_date)
        
        # Set date range
        max_date = pd.to_datetime('2017-11-01')
        min_date = pd.to_datetime('2017-06-30')
        
        try:
            # Retrieve Sunset datetime
            sunset_select = sun_df.loc[weather_date.date() - pd.DateOffset(days=1)]
            sunset_select = sunset_select.loc[sunset_select['station_id'] == weather_station]
            sunset = sunset_select['sunset'][0]

            # Retrieve Sunrise datetime
            sunrise_select = sun_df.loc[weather_date.date()]
            sunrise_select = sunrise_select.loc[sunrise_select['station_id'] == weather_station]
            sunrise = sunrise_select['sunrise'][0]

            print('-')
            print('Sunset UTC DateTime:', sunset)

            print('Sunrise UTC DateTime:', sunrise)

            if (weather_date >= sunset and weather_date <= sunrise):
                print('nightime : append')
                weather_sample = weather_df.loc[weather_df['utc_date'] == weather_date]
                weather_sample = weather_sample.loc[weather_sample['station_id'] == weather_station]
                print('weather_sample:', type(weather_sample))
                weather_night = weather_night.append(weather_sample)
                #weather_night.append([index])
            else:
                print('daytime : null')

            print('\n')
        except:
            print('Exception')
    return weather_night

In [36]:
weather_night = light_weather(sun_df, weather_df)
#save_results(weather_night)

*****
Station : KMTCORVA9 Index : 0
Weatherstation DateTime: 2017-07-01 06:12:00
-
Sunset UTC DateTime: 2017-07-01 03:31:54
Sunrise UTC DateTime: 2017-07-01 11:48:49
nightime : append
weather_sample: <class 'pandas.core.frame.DataFrame'>




### Read Result

In [21]:
csv_weather_night = pd.read_csv('../output/weather_night-180801.csv')

In [32]:
print('weather_df        :', len(weather_df))
print('csv_weather_night :', len(csv_weather_night))
print('difference        :', len(weather_df) - len(csv_weather_night))

weather_df        : 727764
csv_weather_night : 325052
difference        : 402712
